In [1]:
import qiskit 


In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
# from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
# from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
# service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [3]:
from qiskit import *
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import math

In [4]:
def adder(qc, a, b, c, m):
    
    qc.reset(c)
    
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 1):
        # print(i)
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i+1])
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(a[m - 1], b[m - 1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(c[m-1], b[m-1], b[m])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #
    qc.cx(a[m - 1], b[m - 1])
    qc.cx(c[m - 1], b[m - 1])
    
    for i in range(m - 2, -1, -1):
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(c[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i + 1])
        
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(a[i], b[i])
        qc.cx(c[i], b[i])


In [5]:
# addInverse(mod, b)
def inverseAdder(qc, a, b, c, m):
   
    qc.reset(c)
    
    for i in range(m - 1):
        #These two operations act as a sum gate; if a control bit is 
        #in the |1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(c[i], b[i])
        qc.cx(a[i], b[i])
        
        #Reversing the gate operations performed during the carry gate
        #implementations, which is done to reset all carry bits to 
        #the |0> state
        qc.ccx(a[i], b[i], c[i + 1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i + 1])
    
    #
    qc.cx(c[m - 1], b[m - 1])
    qc.cx(a[m - 1], b[m - 1])
    
    #Reversing the gate operation performed on b[n-1]
    qc.cx(a[m - 1], b[m - 1])
    
    #For the last iteration of the carry gate, instead of feeding the
    #result to c[n], we use b[n], which is why c has only n bits,
    #with c[n-1] being the last carry bit
    qc.ccx(c[m-1], b[m-1], b[m])
    qc.cx(a[m-1], b[m-1])
    qc.ccx(a[m - 1], b[m - 1], b[m])
        
        
    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) 
    #with output fed to c[i+1]
    for i in range(m - 2, -1, -1):
        # print(i)
        qc.ccx(c[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(a[i], b[i], c[i+1])


In [6]:
def assign(qc: QuantumCircuit, q, c):
    qc.reset(q)
    
     # Initializing |a> qubits according to input values
    i = 0
    while(c != 0):
        if(c % 2 == 1):
            qc.x(q[i])
            # print(1)
        c = math.floor(c/2)
        # print(x)
        i = i + 1

def sizeReq(a):
    i = 0
    while(a != 0):
        a = math.floor(a/2)
        i = i + 1
    
    return i
    

In [8]:
def modAdder(x, y, N): 
    t = min(x, y)
    y = max(x, y)
    x = t
   
    n = sizeReq(x)
    bLen = max(sizeReq(y), 1)
    m = sizeReq(N)
    
    m = max(n, m, bLen)
    # print(n)
    print(m)
    
    a = QuantumRegister(m, 'a') #First number
    b = QuantumRegister(m + 1, 'b') #Second number and sum
    c = QuantumRegister(m, 'c') #Carry bits
    mod = QuantumRegister(m, 'mod') #N 
        
    cl = ClassicalRegister(m + 1, 'cl') #Final output
    flag = ClassicalRegister(1, 'flag') 
    
    temp = QuantumRegister(1, 'temp') #Temprorary Qubit
    
    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(a, b, c, mod, temp, cl, flag)
    
    # initialization of qubits
    assign(qc, a, x)
    assign(qc, b, y)
    assign(qc, mod, N)
    
    # modular addition begins here
    adder(qc, a, b, c, m)
    
    # how is this working 
    with qc.while_loop((flag, 0)):
        qc.reset(temp)
        inverseAdder(qc, mod, b, c, m)
    
        qc.cx(b[m], temp)
        qc.measure(temp, flag)
        print("Ashish")
        # print(flag)

    # adding mod to the negative number
    adder(qc, mod, b, c, m)

    # Measuring all the qubits
    for i in range(m + 1):
        qc.measure(b[i], cl[i])
    print(qc)
    
    #chosing backend and executing job
    backend = AerSimulator()
    
    # First we have to transpile the quantum circuit 
    # to the low-level QASM instructions used by the 
    # backend
    qc_compiled = transpile(qc, backend)
    
    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend.run(qc_compiled, shots=2)
    
    # Grab the results from the job.
    result_sim = job_sim.result()
    
    counts = result_sim.get_counts(qc_compiled)
    print(counts)
        
modAdder(1, 0, 2)

2
Ashish
                                                                              »
   a_0: ─|0>─────────────■─────────■──────────────────────────────────■────■──»
                         │         │                                  │    │  »
   a_1: ─|0>────────■────┼────■────┼──────────────■────■──────────────┼────┼──»
             ┌───┐  │    │    │  ┌─┴─┐            │    │            ┌─┴─┐  │  »
   b_0: ─|0>─┤ X ├──┼────■────┼──┤ X ├──■─────────┼────┼─────────■──┤ X ├──■──»
             └───┘  │    │  ┌─┴─┐└───┘  │       ┌─┴─┐┌─┴─┐┌───┐  │  └───┘  │  »
   b_1: ─|0>────────■────┼──┤ X ├───────┼────■──┤ X ├┤ X ├┤ X ├──┼─────────┼──»
                  ┌─┴─┐  │  └───┘       │  ┌─┴─┐└───┘└───┘└─┬─┘  │         │  »
   b_2: ─|0>──────┤ X ├──┼──────────────┼──┤ X ├────────────┼────┼─────────┼──»
                  └───┘  │              │  └─┬─┘            │    │         │  »
   c_0: ─|0>─────────────┼──────────────■────┼──────────────┼────■─────────┼──»
                       ┌─┴─┐   